## KSQL Aggregates

KSQL provides a number of useful aggregate functions, such as `MAX`/`MIN`, `SUM`, `COUNT` and
others.

In this exercise, we're going to see how we can create aggregated Tables from our KSQL queries.

First, we're going to create a stream for ClickEvents.

```
CREATE STREAM clickevents
  (email VARCHAR,
   timestamp VARCHAR,
   uri VARCHAR,
   number INTEGER)
  WITH (KAFKA_TOPIC='com.udacity.streams.clickevents',
        VALUE_FORMAT='JSON');
```

### `SUM`

Let's first see how we can summarize clickevents by uri.

```
SELECT uri, SUM(number)
FROM clickevents
GROUP BY uri;
```

When we run this query we will receive an output list that aggregates, by number, the total count
of number by amount, to date, for that URI.

You will notice that values continue to print out to the screen on a periodic basis -- thats ok,
that just means the table-based representation is updating. You could instead create a table and
then periodically query that table to view updates.

### `HISTOGRAM`

Another useful function is `HISTOGRAM`, which allows us to count the number of occurrences for a
given value over a period of time. In this example, lets update the previous example with a
histogram, so not only will we have the `SUM` of all number, but also the number of times we've
had a uri reported at all.

```
SELECT uri,
  SUM(number) AS total_number,
  HISTOGRAM(uri) AS num_uri,
  COUNT(uri) as count_uri
FROM clickevents
GROUP BY uri;
```

Output:

```
https://www.sanchez.com/terms.html | 739 | {https://www.sanchez.com/terms.html=1} | 1
http://lester.com/ | 131 | {http://lester.com/=2} | 2
https://www.walker.com/blog/blog/homepage/ | 183 | {https://www.walker.com/blog/blog/homepage/=1} | 1
http://cooper.com/faq/ | 311 | {http://cooper.com/faq/=2} | 2
http://www.michael.com/explore/wp-content/post.php | 705 | {http://www.michael.com/explore/wp-content/post.php=1} | 1
```

For this simple case it seems easier to just use `COUNT`.

### `TOPK`

Another common usage of stream processing is to find the top number of some value in a window.

Let's define a tumbling time window of 5 minutes, and select the top 5 numbers by uri.

```
SELECT uri , TOPK(number, 5)
FROM clickevents
WINDOW TUMBLING (SIZE 30 SECONDS)
GROUP BY uri;
```

You'll see the window begin to scroll by. As the top 5 numbers by uri updates for each of our
currencies, the query will update. If you wait for 5 minutes (?? this should be 30 seconds I think), you will see the window reset.

Output:

```
https://snyder.net/ | [698, 610, 591]
http://cook-robinson.net/search/terms.html | [897, 205, 103]
http://soto.com/ | [203]
https://howard-blankenship.biz/tag/faq/ | [113]
https://harvey.com/ | [903]
https://www.johnson-wiggins.com/explore/search/ | [80]
https://www.clark-campbell.org/categories/tags/privacy/ | [754]
http://www.roach.net/posts/privacy.php | [656]
http://little-parker.com/search/search/ | [778]
http://www.rose.biz/faq/ | [278]
http://edwards.com/main/category/main.asp | [506]
```